# Summary report in a workbook

We want to create a workbook with:

- Authoring information and instruction
- Summary table for species with links
- Trait codes and descriptions
- Vocabularies
- List of references


## Setup

### Import modules

In [1]:
# work with paths in operating system
from pathlib import Path
import os

# work with xlsx workbooks
import openpyxl
from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.styles import Alignment, PatternFill, Border, Font # Side, Alignment, Protection,
from openpyxl.formatting import Rule
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.worksheet.datavalidation import DataValidation

from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter

# For database connection
from configparser import ConfigParser
import psycopg2
from psycopg2.extras import DictCursor

# Pandas for calculations
import pandas as pd


### Define paths for input and output

In [2]:
repodir = Path("../../") 
inputdir = repodir / "data" / "output-report"
os.listdir(inputdir)

FileNotFoundError: [Errno 2] No such file or directory: '../../data/output-report'

### Database connection

Function to parse connection parameters from a file

In [ ]:
def read_dbparams(filename,section="postgresql"):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db


Reading the default parameters for this session:

In [ ]:
filename = repodir / 'secrets' / 'database.ini'
dbparams=read_dbparams(filename,section='aws-lght-sl')

In [ ]:
if "conn" not in globals() or conn.closed!=0:
    print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(**dbparams)
if "cur" not in globals() or cur.closed:
    cur = conn.cursor(cursor_factory=DictCursor)

## Create workbook

### Styles
Define styles to be used across the workbook

In [ ]:
cent_align=Alignment(horizontal='center', vertical='center', wrap_text=False)
wrap_align=Alignment(horizontal='left', vertical='top', wrap_text=True)

sheet_colors = {"intro": "1072BA" , "summary": "5AFF5A", "default":"505050", "addentry": "20CA82"}

table_style={"Instructions":TableStyleInfo(name="TableStyleMedium9", showFirstColumn=True, showLastColumn=False, 
                                           showRowStripes=True, showColumnStripes=False),
             "Contributor": TableStyleInfo(name="TableStyleMedium18", showFirstColumn=True,
                       showLastColumn=False, showRowStripes=False, showColumnStripes=False),
             "Lists": TableStyleInfo(name="TableStyleMedium14", showFirstColumn=True,
                       showLastColumn=False, showRowStripes=False, showColumnStripes=False),
             "Info":  TableStyleInfo(name="TableStyleMedium14", showFirstColumn=True,
                       showLastColumn=False, showRowStripes=False, showColumnStripes=False),
             "Vocabularies": TableStyleInfo(name="TableStyleMedium14", showFirstColumn=True,
                       showLastColumn=False, showRowStripes=False, showColumnStripes=False),
             "Entry": TableStyleInfo(name="TableStyleMedium18", showFirstColumn=False,
                       showLastColumn=False, showRowStripes=False, showColumnStripes=False)

             }




In [ ]:
wb = Workbook()

In [ ]:
wsheets = (
    {"title": "About", "colWidths":[("A",90),("B",40)], "tabColor":"intro","active":True},
    {"title": "Summary", "colWidths":[("A",70),("B",10),(("C","D","E","F","G"),30)], "tabColor":"summary"},
    {"title": "References", "colWidths":[("A",30),("B",60)], "tabColor":"addentry"},
    {"title": "Trait description", "colWidths":[("A",12),("B",30),("C",70)], "tabColor":"default"}
    )
for item in wsheets:
    if "active" in item.keys():
        ws = wb.active
        ws.title = item['title']
    else:
        ws = wb.create_sheet(item['title'])
    for k in item['colWidths']:
        for j in k[0]:
            ws.column_dimensions[j].width = k[1]
    ws.sheet_properties.tabColor = sheet_colors[item["tabColor"]]


In [ ]:
ws = wb["About"]

info = ("Export from Fireveg Database",
 "Database design by David A. Keith and José R. Ferrer-Paris",
 "Centre for Ecosystem Science / University of New South Wales")

for row in info:
    ws.append((row,))
ws.protection.sheet = True

In [ ]:
cur.execute("SELECT code,name,description,value_type,life_stage,life_history_process,priority FROM litrev.trait_info ORDER BY code")
trait_info = cur.fetchall()

ws = wb["Trait description"]
ws.append(["Trait Code", "Trait Name", "Description", "Type", "Life stage", "Life history process", "Priority"])

for row in trait_info:
    ws.append(row)
    
#ws.max_row
for k in range(2,ws.max_row+1):
    ws.cell(k,3).alignment=wrap_align
    
tab = Table(displayName="TraitInformation", ref="A1:G{}".format(ws.max_row))

tab.tableStyleInfo = table_style["Info"]
ws.add_table(tab)
ws.protection.sheet = True

In [ ]:
def summarise_values(x,w):
    if None in x:
        sfx = "*"
    else:
        sfx = ""
    df=pd.concat({"value": pd.Series(x),"weight": pd.Series(w)},axis=1)
    res = df.groupby(by="value").sum() / df.weight.sum()
    res = res.sort_values(by="weight",ascending=[0])
    val = ""
    glue = ""
    for index, row in res.iterrows():
        if row['weight'] > 0.1:
            val = val + glue + index 
            glue = " / "
        elif row['weight'] > 0.05:
            val = val + glue + ("(%s)" % index) 
            glue = " / "
        else:
            val = val + glue + ("[%s]" % index)
            glue = " / "
    return val + sfx

In [ ]:
for trait in ['surv1','surv4','repr2','germ1','rect2']:
    cur.execute("SELECT species,species_code,array_agg(norm_value::text) as val,array_agg(weight) as w from litrev.{} WHERE species ilike '%euca%' GROUP BY species, species_code limit 38".format(trait))
    res = cur.fetchall()
    df1 = pd.DataFrame(res)
    col1="%s.v" % trait
    col2="%s.w" % trait
    
    df1=df1.rename(columns={0:"Species",1:"Code",2:col1,3:col2})
    df1[trait]=df1.apply(lambda row : summarise_values(row[col1],row[col2]), axis = 1)
    if "df" in globals():
        df = pd.merge(df, df1, on = ["Species","Code"], how = "outer").sort_values(by="Species",ascending=[1])
    else:
        df = df1

In [ ]:
df

In [ ]:
#df

In [ ]:

ws = wb["Summary"]
ws.append(['Species','Code','surv1','surv4','repr2','germ1','rect2'])
rows = dataframe_to_rows(df[['Species','Code','surv1','surv4','repr2','germ1','rect2']],index=False, header=False)

for r_idx, row in enumerate(rows, 2):
    for c_idx, value in enumerate(row, 1):
        ws.cell(row=r_idx, column=c_idx, value=value)

tab = Table(displayName="Summary", ref="A1:{}{}".format(get_column_letter(c_idx),r_idx))
tab.tableStyleInfo = table_style["Lists"]
ws.add_table(tab)


In [ ]:
wb.save(inputdir / "fireveg-trait-report-model.xlsx")

In [ ]:
cur.close()
        
if conn is not None:
    conn.close()
    print('Database connection closed.')